In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from pandas import DataFrame, concat
from pandas import DataFrame
import matplotlib.pyplot as plt
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import itertools
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from joblib import parallel_backend
from sklearn.model_selection import GridSearchCV
import sys
import os
import torch
from torch import nn
import math
import matplotlib.pyplot as plt


In [45]:
torch.manual_seed(111)


# Preparing Harvard

In [46]:
target = 'fclinra08'

# missing: gold stage, cough, sputum, wheeze, baseline fev1, baseline fev1% predicted, symptoms wake you up at night, 2 ore more exacerbations in the last year, asthma, needs help with daily activities, chronic kidney disease, icu last year, hospitalized last year, smoker, pill for exa, inhaler
relevant_vars = ['dem02', 'dem03', 'bclinra01', 'bclinra02', 'bclinpt04', 'bclinpt15', 'mmrc', 'bclinpt07', 'bclinpt34',  'bclinpt28',  'login', 'bclinpt19', 'bclinpt08','bclinpt09','bclinpt38', 'bclinpt32','bclinra11','bclinra07','psqi09','sgrq02','sgrq03', 'exact4', 'exact3', 'sgrq05','sgrq09', 'bclinra101', 'bclinra202c_2', 'bclinra202c_5', 'psqi06', 'bclinpt14','SRPSAT49_Bank','bclinpt11' ,'bclinpt26', 'bclinpt24','bclinpt43', 'bclinpt44','bclinpt45','bclinra06','bclinra06_1024','bclinra06_512','bclinra06_256','bclinra06_128','bclinra06_64','bclinra06_32','bclinra06_16','bclinra06_8','bclinra06_4','bclinra06_2','bclinra06_1', 'exact2','bclinpt22', 'bclinpt06', target] # (cancer), 'bclinpt36' fast breathing, 

# load harvard dataset
harvard = pd.read_csv('./harvard/data/dropped_variables.csv', sep=";")
harvard = harvard[relevant_vars]

# change mmrc by starting at 0 instead of 1
harvard['mmrc'] = harvard['mmrc'] - 1

# make female 0 and male = 1
harvard.loc[harvard['dem03'] == 2, 'dem03'] = 0
harvard.loc[harvard['dem03'] == 1, 'dem03'] = 1

# rename columns to common names with exaggerate
harvard = harvard.rename(columns={
    'dem02':'age', 
    'dem03': 'gender', 
    'bclinpt19': 'cancer', 
    'bclinpt04': 'hypertension', 
    'bclinpt15': 'diabetes', 
    'bclinpt07': 'heart_failure', 
    'bclinpt06': 'ischemic_heart_disease',
    target: 'target', 
    'bclinra01': 'height', 
    'bclinra02': 'weight', 
    'bclinpt34': 'fever',  
    'bclinpt08':'myocardial_infarct', 
    'bclinpt09':'stroke', 
    'bclinpt28':'dyspnoea', 
    'bclinpt36': 'fast_breathing', 
    'mmrc': 'mmrc', 
    'bclinpt22': 'prev_exacerb',
    'bclinpt38': 'wheezing',
    'sgrq05': 'wheezing_attacks',
    'sgrq09': 'wheeze_morning', 
    'bclinpt32': 'sputum',
    'sgrq03': 'bring_up_sputum',
    'exact4': 'how_difficult_sputum',
    'exact3': 'how_much_sputum',
    'bclinra11': 'gold',
    'exact2': 'cough_today',
    'psqi09': 'cough_snore',
    'sgrq02': 'cough_four_weeks',
    'sgrq20': 'cough_hurts',
    'sgrq21': 'cough_tired',
    'bclinra101': 'fev1',
    'bclinra202c_2': 'percent_fev1_pred',
    'bclinra202c_5': 'percent_fev1_pred_2',
    'psqi06': 'waking_up_at_night',
    'bclinpt14': 'asthma',
    'bclinpt11': 'chronic_kidney_disease',
    'bclinpt26': 'icu_last_year',
    'bclinpt24': 'hospitalizations_last_year',
    'bclinpt43': 'medication_copd',
    'bclinpt44': 'other_antibiotic',
    'bclinpt45': 'other_medication',
    'bclinra06': 'medication_copd_2',
    'bclinra06_1024': 'other_copd_medication',
    'bclinra06_512': 'qvar',
    'bclinra06_256': 'azmacort',
    'bclinra06_128': 'pulmicort',
    'bclinra06_64': 'flovent',
    'bclinra06_32': 'foradil',
    'bclinra06_16': 'serevent',
    'bclinra06_8': 'spiriva',
    'bclinra06_4': 'combivent',
    'bclinra06_2': 'atrovent',
    'bclinra06_1': 'proventil',
 }) 

# generate the cough variable
harvard['cough'] = 0

# if cought_today is 2,3,4 or 5 set cough to 1
harvard.loc[harvard['cough_today'] > 1, 'cough'] = 1

# TODO check if we should use this variable. Seems like it might be related to snoring only
# if cough_snore is > 1 set cough to 1
# harvard.loc[harvard['cough_snore'] > 1, 'cough'] = 1
# if cough_four_weeks is < 4 set cough to 1
harvard.loc[harvard['cough_four_weeks'] < 4, 'cough'] = 1
# drop cough_today, cough_snore, cough_four_weeks
harvard = harvard.drop(columns=['cough_today', 'cough_snore', 'cough_four_weeks'], axis=1)

# TODO we could use other sputum variables as well
# if 'sputum' is 0 set it to 1 and the other way around
harvard['sputum'] = 1 - harvard['sputum']

# if 'sputum' is na and bring_up_sputum is < 4 set it to 1
harvard.loc[harvard['sputum'].isna() & (harvard['bring_up_sputum'] < 4), 'sputum'] = 1

# drop bring_up_sputum, how_difficult_sputum, how_much_sputum
harvard = harvard.drop(columns=['bring_up_sputum', 'how_difficult_sputum', 'how_much_sputum'], axis=1)

# reconstruct wheezing from sgrq05, bclinpt38, sgrq09
harvard['wheezing'] = 1 - harvard['wheezing']

# if wheezing is na and wheezing_attacks is < 4 set it to 1
harvard.loc[harvard['wheezing'].isna() & (harvard['wheezing_attacks'] < 4), 'wheezing'] = 1
harvard.loc[harvard['wheezing'].isna() & (harvard['wheeze_morning'] == 2 ), 'wheezing'] = 1
harvard = harvard.drop(columns=['wheezing_attacks', 'wheeze_morning'], axis=1)

# calculate 2 or more exacerbations last year variable
harvard['two_exacerbations_last_year'] = harvard['prev_exacerb']
harvard.loc[harvard['two_exacerbations_last_year'] <= 2, 'two_exacerbations_last_year'] = 0
harvard.loc[harvard['two_exacerbations_last_year'] > 2, 'two_exacerbations_last_year'] = 1
harvard = harvard.drop(columns=['prev_exacerb'], axis=1)
# TODO maybe we have to use a different exacerbation definition

# if patient was to the icu at least one time
harvard['icu'] = harvard['icu_last_year']
harvard.loc[harvard['icu'] == 1, 'icu'] = 0
harvard.loc[harvard['icu'] > 1, 'icu'] = 1
harvard = harvard.drop(columns=['icu_last_year'], axis=1)

# reconstruction hospital from bclinpt26 (icu), bclinpt27 (icu), bclinpt24, bclinpt25
harvard['hospitalization'] = harvard['hospitalizations_last_year']
harvard.loc[harvard['hospitalization'] == 1, 'hospitalization'] = 0
harvard.loc[harvard['hospitalization'] > 1, 'hospitalization'] = 1
harvard.loc[harvard['hospitalization'].isna() & (harvard['icu'] == 1), 'hospitalization'] = 1
harvard = harvard.drop(columns=['hospitalizations_last_year'], axis=1)
# don't use daily activities as data is not sufficient

# create smoker variable (everyone)
harvard['smoker'] = 1

# create inhaler variable
harvard['inhaler'] = 0
harvard.loc[(harvard['qvar'] == 1) | (harvard['azmacort'] == 1) | (harvard['flovent'] == 1)| (harvard['foradil'] == 1)| (harvard['serevent'] == 1)| (harvard['spiriva'] == 1)| (harvard['combivent'] == 1)| (harvard['atrovent'] == 1) | (harvard['proventil'] == 1)| (harvard['pulmicort'] == 1), 'inhaler'] = 1
# drop inhaler columns
harvard = harvard.drop(columns=['qvar', 'azmacort', 'flovent', 'foradil', 'serevent', 'spiriva', 'combivent', 'atrovent', 'proventil', 'pulmicort'], axis=1)

# create pill for exa variable
    # bclinpt43 = 2048, 1024, 4096, 
    # bclinpt44 any
harvard['pill_for_exa'] = 0
harvard.loc[(harvard['medication_copd'] == 2048) | (harvard['medication_copd'] == 1024)| (harvard['medication_copd'] == 4096), 'pill_for_exa'] = 1
harvard = harvard.drop(columns=['medication_copd', 'other_antibiotic','other_medication', 'medication_copd_2', 'other_copd_medication' ], axis=1)

# if percent_fev1_pred is na take percent_fev1_pred_2
harvard['percent_fev1_pred'] = harvard['percent_fev1_pred'].fillna(harvard['percent_fev1_pred_2'])

# TODO daily activities restricted var

# drop all other not any more relevant vars
harvard = harvard.drop(columns=['SRPSAT49_Bank', 'percent_fev1_pred_2', 'bclinra07', 'cancer', 'dyspnoea', 'fever', 'myocardial_infarct', 'stroke', 'fev1'])


# get the baseline dataset
unique_logins = harvard['login'].unique()
# baseline dataframe has the same columns as the original dataset
baseline = []
for login in unique_logins:
    # add the first row of the selection to the baseline dataset
    baseline.append(harvard[harvard['login'] == login].iloc[0])
# turn lists into dataframes but with columns from harvard dataset
harvard = pd.DataFrame(baseline, columns=harvard.columns)
harvard = harvard.drop('login', axis=1)

# swap variable for which yes and no are reversed (yes = 0, no = 1)
harvard['hypertension'] = 1 - harvard['hypertension']
# harvard['fever'] = 1 - harvard['fever']
harvard['heart_failure'] = 1- harvard['heart_failure']

# subtract 1 of all target values and revert yes and no
harvard['target'] = harvard['target'] - 1
harvard['target'] = 1 - harvard['target']

# calculate bmi from weight and height
harvard['bmi'] = harvard['weight'] / (harvard['height'] / 100) ** 2
harvard = harvard.drop(columns=['weight', 'height'])
harvard = harvard.reindex(sorted(harvard.columns), axis=1)

# print sorted columns
# print(sorted(harvard.columns))
# print(sorted(triage.columns))

# # if prev_exacerb is 1 set it to 0
# harvard.loc[harvard['prev_exacerb'] == 1, 'prev_exacerb'] = 0
# # if prev_exacerb is above 1 set it to 1
# harvard.loc[harvard['prev_exacerb'] > 1, 'prev_exacerb'] = 1
# # plot_hist(harvard)

##########
# Missing Values & duplicates
##########

# drop the records where target is na
harvard = harvard.dropna(subset=['target'])

# print missing values per variable and total
print("missing values per variable")
print("total records: "+str(len(harvard)))
print(harvard.isnull().sum())

# drop all records where target is nan
harvard = harvard.dropna(subset=['target'])

numerical_vars = ['age', 'bmi','percent_fev1_pred']
for var in numerical_vars:
    harvard[var] = harvard[var].fillna(harvard[var].median())



# impute gender, cancer, hypertension, diabetes, heart_failure, target with most frequent value
categorical_vars = ['asthma','chronic_kidney_disease', 'cough', 'diabetes',  'gender', 'gold', 'heart_failure', 'hospitalization', 'hypertension', 'icu', 'inhaler', 'ischemic_heart_disease', 'mmrc', 'pill_for_exa', 'smoker', 'sputum', 'target', 'two_exacerbations_last_year', 'waking_up_at_night', 'wheezing'] # , 'fast_breathing',,, 'prev_exacerb' 'dyspnoea', 'cancer', 'myocardial_infarct', 'stroke', 'fever',
for var in categorical_vars:
    harvard[var] = harvard[var].fillna(harvard[var].mode()[0])


# print("empty cells remaining in the dataset")
# print(harvard.isnull().sum().sum())

# print("mv imputation")

# plot_hist(harvard)

##########
# Feature engineering
############

# print("feature engineering")

# plot_hist(harvard)

##########
# Normalization
##########

numeric_vars = ['age', 'bmi', 'mmrc', 'percent_fev1_pred']
df_nr = harvard[numeric_vars]
df_rest = harvard.drop(columns=numeric_vars)
transf = MinMaxScaler(feature_range=(0, 1), copy=True).fit(df_nr)
tmp = DataFrame(transf.transform(df_nr), index=harvard.index, columns=numeric_vars)
harvard= concat([tmp, df_rest], axis=1)

###########
# Target variable
###########


# use smote to rebalance the harvard dataset in place
# print("perform smote")

sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(harvard.drop(columns=['target']), harvard['target'])
X_res['target'] = y_res
harvard = X_res

# use undersampling

# # get number of underrespresented class 0
# u = harvard['target'].value_counts().min()
# u_label = harvard['target'].value_counts().idxmin()

# # overrepresented class 1
# o = harvard['target'].value_counts().max()
# o_label = harvard['target'].value_counts().idxmax()

# # add all underrepresented samples to a new dataframe
# underrepresented = harvard[harvard['target'] == u_label]
# # sample u number of overrepresented samples from the dataset
# overrepresented = harvard[harvard['target'] == o_label].sample(n=u, random_state=42)
# # concatenate underrepresented and overrepresented
# harvard = pd.concat([underrepresented, overrepresented])
# # plot_hist(harvard)
# print(harvard['target'].value_counts())

# sort the columns
harvard = harvard.reindex(sorted(harvard.columns), axis=1)
# convert all variables to int
harvard = harvard.astype(int)

# # save dataset
harvard.to_csv('../harvard/data/triage_cv.csv', index=False, sep=';')

C:\Users\Daniel\AppData\Local\Temp\ipykernel_2464\1621371962.py:7: DtypeWarning: Columns (24,157,481,483,484,486,487,503) have mixed types. Specify dtype option on import or set low_memory=False.
  harvard = pd.read_csv('../harvard/data/dropped_variables.csv', sep=";")


missing values per variable
total records: 150
age                             0
asthma                          9
bmi                             7
chronic_kidney_disease          9
cough                           0
diabetes                        9
gender                          0
gold                           28
heart_failure                   9
hospitalization                 1
hypertension                    8
icu                             1
inhaler                         0
ischemic_heart_disease          9
mmrc                            0
percent_fev1_pred              31
pill_for_exa                    0
smoker                          0
sputum                          0
target                          0
two_exacerbations_last_year     0
waking_up_at_night              0
wheezing                        1
dtype: int64


# Preparing Triage

In [47]:
triage = pd.read_excel('./triage/data/data.xlsx')

# remove all empty rows
triage = triage.dropna(how='all')

# drop Alternate Diagnosis? (y/n), Final Triage 1 (1-4), Confidence 1 (%), Final Triage 2 (1-4), Confidence 2 (%), Highly Unrealistic Case (Y), Highly Uncertain (Y), Profile Severity (1-5)
triage = triage.drop(columns=['Unnamed: 0', 'Alternate Diagnosis? (y/n)', 'Final Triage 1 (1-4)', 'Confidence 1 (%)', 'Final Triage 2 (1-4)', 'Confidence 2 (%)', 'Highly Unrealistic Case (Y)', 'Highly Uncertain (Y)', 'Profile Severity (1-5)'], axis=1)
# reset index
triage = triage.reset_index(drop=True)


triage = triage.drop(triage.columns[triage.columns.get_loc('Exacerbation (Y/N)') + 1:], axis=1)
# drop Vitals Severity (1-5)
triage = triage.drop(columns=['Vitals Severity (1-5)'], axis=1)
# convert all values where it say 'Unknown' to NaN
triage = triage.replace('Unknown', np.nan)

# turn Male into 1 and Female into 0
triage = triage.replace('Male', 1)
triage = triage.replace('Female', 0)

# convert height from Height(ft) and Height(in) to Height(cm)
triage['Height(cm)'] = triage['Height(ft)'] * 30.48 + triage['Height(in)'] * 2.54
# drop Height(ft) and Height(in)
triage = triage.drop(columns=['Height(ft)', 'Height(in)'], axis=1)

# convert weight from Weight(lbs) to Weight(kg)
triage['Weight(kg)'] = triage['Weight (lb)'] * 0.45359237
# drop Weight(lbs)
triage = triage.drop(columns=['Weight (lb)'], axis=1)

# recalculate BMI for metric system
triage['BMI'] = triage['Weight(kg)'] / (triage['Height(cm)'] / 100) ** 2

# convert temperature from Temperature(F) to Temperature(C)
triage['Current Temperature'] = (triage['Current Temperature'] - 32) * 5 / 9


risk_factors = ()
# add all unique values in columns Risk Factor 1-4 to risk_factors, make sure there are no duplicates
for i in range(1, 5):
    risk_factors += tuple(triage['Risk Factor ' + str(i)].unique())
risk_factors = tuple(set(risk_factors))
# remove NaN from risk_factors
risk_factors = tuple(filter(lambda x: x == x, risk_factors))
# create a column for each risk factor in risk_factors
for risk_factor in risk_factors:
    triage[risk_factor] = 0
# set all values in the columns to 1 if the risk factor is present
for i in range(1, 5):
    for risk_factor in risk_factors:
        triage.loc[triage['Risk Factor ' + str(i)] == risk_factor, risk_factor] = 1
# drop Risk Factor 1-4
triage = triage.drop(columns=['Risk Factor ' + str(i) for i in range(1, 5)], axis=1)

medications = ()
# split each value in 'Current Medication' at + and add the medication to the tuple if it is not already in there
for medication in triage['Current Medication']:
    # if the value is not empty
    if medication == medication:
        # split the value at + and add the medication to the tuple if it is not already in there
        for med in str(medication).split('+'):
            if med not in medications:
                medications += (med,)



# create a column for each medication in medications
for medication in medications:
    triage[medication] = 0
# set all values in the columns to 1 if the medication is present
for medication in medications:
    triage.loc[triage['Current Medication'].str.contains(medication, na=False), medication] = 1
# drop Current Medication
triage = triage.drop(columns=['Current Medication'], axis=1)

# turn Baseline Dyspnea (MMRC) into the number that the value contains
triage['Baseline Dyspnea (MMRC)'] = triage['Baseline Dyspnea (MMRC)'].str.extract('(\d+)')
# turn COPD Gold Stage into the number that the value contains
triage['COPD Gold Stage'] = triage['COPD Gold Stage'].str.extract('(\d+)')
# turn Recent Worsening in Symptoms? into categorical values for each unique values
triage['Recent Worsening in Symptoms?'] = triage['Recent Worsening in Symptoms?'].astype('category').cat.codes
# turn % Controller Medication Taken Over Last Week into categorical values for each unique values
triage['% Controller Medication Taken Over Last Week'] = triage['% Controller Medication Taken Over Last Week'].astype('category').cat.codes
# turn Short of Breath? into categorical values for each unique values
triage['Short of Breath?'] = triage['Short of Breath?'].astype('category').cat.codes
# turn Cough? into categorical values for each unique values
triage['Cough?'] = triage['Cough?'].astype('category').cat.codes
# turn Sputum? into categorical values for each unique values
triage['Sputum?'] = triage['Sputum?'].astype('category').cat.codes
# turn Wheezing? into categorical values for each unique values
triage['Wheezing?'] = triage['Wheezing?'].astype('category').cat.codes

# turn Current Dyspnea (MMRC) into the number that the value contains
triage['Current Dyspnea (MMRC)'] = triage['Current Dyspnea (MMRC)'].str.extract('(\d+)')

# turn all values that contain 'No' into 0 and all values that contain 'Yes' into 1
triage = triage.replace('No', 0)
triage = triage.replace('Yes', 1)
triage = triage.replace('Y', 1)
triage = triage.replace('y', 0)
triage = triage.replace('N', 0)

# print missing values
print(triage.isnull().sum())

categorical_vars = ['COPD Gold Stage']
# impute using mode
for var in categorical_vars:
    triage[var].fillna(triage[var].mode()[0], inplace=True)



numerical_vars = ['Baseline FEV1', 'Baseline Heart Rate', 'Baseline Pulse Ox', 'Baseline Heart Rate.1', 'Baseline Pulse Ox.1', 'Current Heart Rate', 'Current Pulse Ox', 'Current Temperature', 'Current FEV1 (% Predicted)', 'Baseline FEV1 (% Predicted)']

# impute using median
for var in numerical_vars:
    triage[var].fillna(triage[var].median(), inplace=True)

# create fever variable
# triage['fever'] = 0
# triage.loc[triage['Current Temperature'] > 38, 'fever'] = 1
# # drop Current Temperature
# triage = triage.drop(columns=['Current Temperature'], axis=1)

# scaling
scaler = MinMaxScaler()
target = triage['Exacerbation (Y/N)']
triage = triage.drop(columns=['Exacerbation (Y/N)'], axis=1)
triage = pd.DataFrame(scaler.fit_transform(triage), columns=triage.columns)
triage['Exacerbation (Y/N)'] = target

# target variable
# print(triage['Exacerbation (Y/N)'].value_counts())
# balance using smote
sm = SMOTE(random_state=42)
X = triage.drop(columns=['Exacerbation (Y/N)'], axis=1)
y = triage['Exacerbation (Y/N)']
X, y = sm.fit_resample(X, y)
triage = X
triage['Exacerbation (Y/N)'] = y

# without current temperature for now

relevant_vars = [
    'Age', 
    'Gender', 
    'BMI', 
    'Baseline Dyspnea (MMRC)', 
    'COPD Gold Stage', 
    'Cough?', 'Sputum?', 'Wheezing?', 
    'Baseline FEV1', 
    # 'Current FEV1 (% Predicted)', 
    'Baseline FEV1 (% Predicted)', 
    'Exacerbation (Y/N)', 
    'Respiratory Symptoms Wake You Up At Night More Than Usual?', 
    '2 or More Exacerbations In Last Year', 
    'High Blood Pressure', 
    'Coronary Artery Disease', 
    'Congestive Heart Failure', 
    'Asthma', 
    'Needs Help Performing Daily Activities', 
    'Chronic Kidney Disease',  
    'Diabetes', 
    'Visited ICU for COPD in Last Year', 
    'Hospitalized for COPD in Last Year', 
    'Smoker', 
    'Pill for Controlling Exacerbations ', 
    ' Prescribed Inhaler ', 
    ' Rescue Inhaler As Needed', 
    'Prescribed Inhaler (Azithromycin or Roflumilast) ', 
    ' rescue Inhaler As Needed.', 
    'Rescue Inhaler Or Nebulizer As Needed For Breathing', ]

triage = triage[relevant_vars]

# rename vars
triage = triage.rename(columns={
    "Age":"age", 
    "Gender":"gender", 
    "BMI":"bmi", 
    "Baseline Dyspnea (MMRC)":"mmrc", 
    "COPD Gold Stage":"gold", 
    "Cough?":"cough", 
    "Sputum?":"sputum", 
    "Wheezing?":"wheezing", 
    "Baseline FEV1":"fev1", 
    "Current Temperature":"temperature", 
    "Current FEV1 (% Predicted)":"percent_fev1_pred_current", 
    "Baseline FEV1 (% Predicted)":"percent_fev1_pred", 
    "Exacerbation (Y/N)":"target", 
    "Respiratory Symptoms Wake You Up At Night More Than Usual?":"waking_up_at_night", 
    "2 or More Exacerbations In Last Year":"two_exacerbations_last_year", 
    "High Blood Pressure":"hypertension", 
    "Coronary Artery Disease":"ischemic_heart_disease", 
    "Congestive Heart Failure":"heart_failure", 
    "Asthma":"asthma", 
    "Needs Help Performing Daily Activities":"daily_activities_restricted", 
    "Chronic Kidney Disease":"chronic_kidney_disease",  
    "Diabetes":"diabetes", 
    "Visited ICU for COPD in Last Year":"icu", 
    "Hospitalized for COPD in Last Year":"hospitalization", 
    "Smoker":"smoker", 
    "Pill for Controlling Exacerbations ":"pill_for_exa", 
    " Prescribed Inhaler ":"medication_inhaler", 
    " Rescue Inhaler As Needed":"medication_rescue_inhaler", 
    "Prescribed Inhaler (Azithromycin or Roflumilast) ":"medication_azithromycin_roflumilast", 
    " rescue Inhaler As Needed.":"medication_rescue_inhaler_as_needed", 
    "Rescue Inhaler Or Nebulizer As Needed For Breathing":"medication_rescue_inhaler_or_nebulizer"})

# perform some feature engineering
triage["inhaler"] = 0
if triage["medication_inhaler"].any() == 1 or triage["medication_rescue_inhaler"].any() == 1 or triage["medication_rescue_inhaler_as_needed"].any() == 1 or triage["medication_rescue_inhaler_or_nebulizer"].any() == 1 or triage["medication_azithromycin_roflumilast"].any() == 1:
    triage["inhaler"] = 1

# drop unnecessary columns
triage = triage.drop(columns=["medication_inhaler", "medication_rescue_inhaler", "medication_rescue_inhaler_as_needed", "medication_rescue_inhaler_or_nebulizer", "medication_azithromycin_roflumilast", "daily_activities_restricted", 'fev1'], axis=1)

# triage['fever'] = 0
# triage.loc[triage['temperature'] > 38, 'fever'] = 1

# sort the columns
triage = triage.reindex(sorted(triage.columns), axis=1)

# export as harvard_cv
triage.to_csv("../triage/data/harvard_cv.csv", index=False, sep=";")

Gender                                                         0
Age                                                            0
BMI                                                            0
Baseline Dyspnea (MMRC)                                        0
COPD Gold Stage                                               32
Baseline FEV1                                                 32
Baseline Heart Rate                                           30
Baseline Pulse Ox                                             36
Recent Worsening in Symptoms?                                  0
% Controller Medication Taken Over Last Week                   0
Short of Breath?                                               0
Cough?                                                         0
Wheezing?                                                      0
Sputum?                                                        0
Current Dyspnea (MMRC)                                         0
Infection?               

c:\Users\Daniel\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
C:\Users\Daniel\AppData\Local\Temp\ipykernel_2464\3482200299.py:73: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  triage.loc[triage['Current Medication'].str.contains(medication, na=False), medication] = 1


# Setting up GAN

In [48]:
# 0 is harvard, 1 is triage


# create train set from triage
harvard_samples = harvard.sample(frac=0.5, random_state=42)
# give all harvard samples the label [0,1]
harvard_samples["dataset"] = 0
triage_samples = triage.sample(frac=0.5, random_state=42)
# give all triage samples the label [1,0]
triage_samples["dataset"] = 1
train = pd.concat([triage_samples.drop(columns=["dataset"]), harvard_samples.drop(columns=["dataset"])])
train = train.reset_index(drop=True).to_numpy()
test = pd.concat([triage_samples["dataset"], harvard_samples["dataset"]])
test = test.reset_index(drop=True)

In [49]:
batch_size = 16
train_loader = torch.utils.data.DataLoader(
    train, batch_size=batch_size, shuffle=True
)


In [56]:
# create a Pytorch discriminator that provides the probability that a sample belongs to the harvard dataset (0) or the triage dataset (1)
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(23, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1),
            nn.Sigmoid(),
        )
    
    def forward(self, x):
        output = self.model(x)
        return output

discriminator = Discriminator()


In [51]:
# create a Pytorch generator that generates a sample (23 variables) that is either from the triage or harvard dataset
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(24, 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, 24)
        self.dropout = nn.Dropout(0.3)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.relu(self.fc3(x))
        return x

# Training the GAN

In [52]:
lr = 0.001
num_epochs = 300
loss_function = nn.BCELoss()
optimizer_discriminator = torch.optim.Adam(discriminator.parameters(), lr=lr)
optimizer_generator = torch.optim.Adam(generator.parameters(), lr=lr)



In [57]:
for epoch in range(num_epochs):
    for n, real_samples in enumerate(train_loader):
        print(real_samples)
        # Data for training the discriminator
        real_samples_labels = torch.ones((batch_size, 1))
        latent_space_samples = torch.randn((batch_size, 2))
        # generated_samples = generator(latent_space_samples)
        generated_samples_labels = torch.zeros((batch_size, 1))
        # all_samples = torch.cat((real_samples, generated_samples))
        all_samples_labels = real_samples_labels
        all_samples = real_samples
        

        # Training the discriminator
        discriminator.zero_grad()
        output_discriminator = discriminator(all_samples)

        loss_discriminator = loss_function(
            output_discriminator, all_samples_labels)

        loss_discriminator.backward()
        optimizer_discriminator.step()


        # # Data for training the generator
        # latent_space_samples = torch.randn((batch_size, 2))

        # # Training the generator
        # generator.zero_grad()
        # generated_samples = generator(latent_space_samples)
        # output_discriminator_generated = discriminator(generated_samples)

        # loss_generator = loss_function(
        #     output_discriminator_generated, real_samples_labels
        # )

        # loss_generator.backward()
        # optimizer_generator.step()


        # Show loss

        if epoch % 10 == 0 and n == batch_size - 1:
            print(f"Epoch: {epoch} Loss D.: {loss_discriminator}")
            # print(f"Epoch: {epoch} Loss G.: {loss_generator}")

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 2.0000, 1.0000,
         0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000,
         1.0000, 1.0000, 0.0000, 0.0000, 0.0000],
        [0.1707, 0.0000, 0.4548, 0.0000, 0.5000, 1.0000, 1.0000, 0.6667, 0.0000,
         0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.4744, 0.0000, 0.0000,
         1.0000, 1.0000, 0.0000, 0.0000, 1.0000],
        [1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 2.0000, 1.0000,
         0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000,
         0.0000, 0.0000, 0.0000, 3.0000, 0.0000],
        [0.0000, 1.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000, 1.0000,
         0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000,
         1.0000, 0.0000, 1.0000, 3.0000, 1.0000],
        [0.1463, 1.0000, 0.5420, 0.0000, 0.0000, 0.0000, 0.0000, 0.6667, 0.0000,
         0.0000, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000, 0.3974, 0.0000

RuntimeError: mat1 and mat2 must have the same dtype